In [ ]:
!nvidia-smi

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import math
import numpy as np
import time
import json
import plotly
import logging
logging.getLogger().setLevel(logging.INFO)

from tqdm import tqdm
from idst_util import trivial
from idst_util import dstc2

from plotly.graph_objs import Scatter, Layout
from plotly.graph_objs.layout import Margin
plotly.offline.init_notebook_mode(connected = True)


# Make sure data is available
trivial.print_idst()
dstc2.check()

# Retrieve raw data
raw_X_train, raw_Y_train, \
raw_X_dev, raw_Y_dev, \
raw_X_test, raw_Y_test, \
ontology = dstc2.retrieve_raw_datasets(train_data_augmentation = False)

In [ ]:
logging.info("+--------------------------------+")
logging.info("|            Baseline            |")
logging.info("+--------------------------------+")

GPU_ID = 1
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")
#DEVICE = "cpu"
if DEVICE == "cpu":
    logging.warning("Running on CPU")
else:
    logging.info("Running on GPU {}".format(GPU_ID))

In [ ]:
logging.info("+--------------------------------+")
logging.info("|          Vocabulary            |")
logging.info("+--------------------------------+")
logging.info("Creating token_to_index, index_to_token and token_to_count dictionaries")

token_to_index = {"<unk>": 0}
index_to_token = {0: "<unk>"}
token_to_count = {"<unk>": 1}

for raw_train_dialog in tqdm(raw_X_train):
    
    for raw_train_turn in raw_train_dialog["turns"]:

        for system_token in raw_train_turn["system"]:
            token = system_token[0]
            if token not in token_to_index:
                token_to_index[token] = len(token_to_index)
                index_to_token[len(token_to_index)] = token
                token_to_count[token] = 1
            else:
                token_to_count[token] += 1
        
        for user_token in raw_train_turn["user"]:
            token = user_token[0]
            if token not in token_to_index:
                token_to_index[token] = len(token_to_index)
                index_to_token[len(token_to_index)] = token
                token_to_count[token] = 1
            else:
                token_to_count[token] += 1
                
assert len(token_to_index) == len(index_to_token)
assert len(token_to_index) == len(token_to_count)

logging.info("Vocabulary length:\t{}".format(len(token_to_index)))

In [ ]:
logging.info("+--------------------------------+")
logging.info("|         Configuration          |")
logging.info("+--------------------------------+")

VOCABULARY_SIZE = len(token_to_index)
EMBEDDING_DIM = 170
HIDDEN_DIM = 100
NUM_NODES = 300
NUM_EPOCHS = 50

METHOD_DIM = len(ontology["method"])
REQUESTED_DIM = int(math.pow(2, len(ontology["requestable"])))
GOAL_FOOD_DIM = len(ontology["informable"]["food"]) + 2 # because of null and dontcare
GOAL_PRICERANGE_DIM = len(ontology["informable"]["pricerange"]) + 2 # because of null and dontcare
GOAL_NAME_DIM = len(ontology["informable"]["name"]) + 2 # because of null and dontcare
GOAL_AREA_DIM = len(ontology["informable"]["area"]) + 2 # because of null and dontcare

logging.info("VOCABULARY_SIZE:\t{}".format(VOCABULARY_SIZE))
logging.info("EMBEDDING_DIM:\t{}".format(EMBEDDING_DIM))
logging.info("HIDDEN_DIM:\t\t{}".format(HIDDEN_DIM))
logging.info("NUM_NODES:\t\t{}".format(NUM_NODES))
logging.info("NUM_EPOCHS:\t\t{}".format(NUM_EPOCHS))
logging.info("METHOD_DIM:\t\t{}".format(METHOD_DIM))
logging.info("REQUESTED_DIM:\t{}".format(REQUESTED_DIM))
logging.info("GOAL_FOOD_DIM:\t{}".format(GOAL_FOOD_DIM))
logging.info("GOAL_PRICERANGE_DIM:\t{}".format(GOAL_PRICERANGE_DIM))
logging.info("GOAL_NAME_DIM:\t{}".format(GOAL_NAME_DIM))
logging.info("GOAL_AREA_DIM:\t{}".format(GOAL_AREA_DIM))

In [ ]:
class LecTrackEncoder(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, vocabulary_size, num_nodes, device):
        super(LecTrackEncoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.device = device
        self.token_embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                            embedding_dim = embedding_dim)
        self.linear = nn.Linear(in_features = (embedding_dim + 1), # because of score
                                out_features = num_nodes)
        self.lstm = nn.LSTM(num_nodes, hidden_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
        embeddings = self.token_embeddings(indices)
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        altered_embeddings = F.relu(self.linear(embeddings_concat_score))
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        return self.hidden
    
model_LecTrackEncoder = LecTrackEncoder(embedding_dim = EMBEDDING_DIM,
                                        hidden_dim = HIDDEN_DIM,
                                        vocabulary_size = VOCABULARY_SIZE,
                                        num_nodes = NUM_NODES,
                                        device = DEVICE)

model_LecTrackEncoder = model_LecTrackEncoder.to(DEVICE)

optimizer_LecTrackEncoder = optim.Adam(model_LecTrackEncoder.parameters(), lr = 1e-4)

In [ ]:
class LecTrackMethodClassifier(nn.Module):
    
    def __init__(self, hidden_dim, method_dim):
        super(LecTrackMethodClassifier, self).__init__()
        self.method_dim = method_dim
        self.linear = nn.Linear(in_features = hidden_dim,
                                out_features = method_dim)
        
    def forward(self, hidden):
        return F.log_softmax(self.linear(hidden).view(-1, self.method_dim), dim = 1)

    
model_LecTrackMethodClassifier = LecTrackMethodClassifier(hidden_dim = HIDDEN_DIM,
                                                          method_dim = METHOD_DIM)

model_LecTrackMethodClassifier = model_LecTrackMethodClassifier.to(DEVICE)

optimizer_LecTrackMethodClassifier = optim.Adam(model_LecTrackMethodClassifier.parameters(), lr = 1e-4)

In [ ]:
class LecTrackRequestedClassifier(nn.Module):
    
    def __init__(self, hidden_dim, requested_dim):
        super(LecTrackRequestedClassifier, self).__init__()
        self.requested_dim = requested_dim
        self.linear = nn.Linear(in_features = hidden_dim,
                                out_features = requested_dim) 
        
    def forward(self, hidden):
        return F.log_softmax(self.linear(hidden).view(-1, self.requested_dim), dim = 1)

model_LecTrackRequestedClassifier = LecTrackRequestedClassifier(hidden_dim = HIDDEN_DIM,
                                                                requested_dim = REQUESTED_DIM)

model_LecTrackRequestedClassifier = model_LecTrackRequestedClassifier.to(DEVICE)

optimizer_LecTrackRequestedClassifier = optim.Adam(model_LecTrackRequestedClassifier.parameters(), lr = 1e-4)

In [ ]:
class LecTrackGoalFoodClassifier(nn.Module):
    
    def __init__(self, hidden_dim, goal_food_dim):
        super(LecTrackGoalFoodClassifier, self).__init__()
        self.goal_food_dim = goal_food_dim
        self.linear = nn.Linear(in_features = hidden_dim,
                                out_features = goal_food_dim)
        
    def forward(self, hidden):
        return F.log_softmax(self.linear(hidden).view(-1, self.goal_food_dim), dim = 1)

model_LecTrackGoalFoodClassifier = LecTrackGoalFoodClassifier(hidden_dim = HIDDEN_DIM,
                                                              goal_food_dim = GOAL_FOOD_DIM)

model_LecTrackGoalFoodClassifier = model_LecTrackGoalFoodClassifier.to(DEVICE)

optimizer_LecTrackGoalFoodClassifier = optim.Adam(model_LecTrackGoalFoodClassifier.parameters(), lr = 1e-4)

In [ ]:
class LecTrackGoalPricerangeClassifier(nn.Module):
    
    def __init__(self, hidden_dim, goal_pricerange_dim):
        super(LecTrackGoalPricerangeClassifier, self).__init__()
        self.goal_pricerange_dim = goal_pricerange_dim
        self.linear = nn.Linear(in_features = hidden_dim,
                                out_features = goal_pricerange_dim)
        
    def forward(self, hidden):
        return F.log_softmax(self.linear(hidden).view(-1, self.goal_pricerange_dim), dim = 1)

model_LecTrackGoalPricerangeClassifier = LecTrackGoalPricerangeClassifier(hidden_dim = HIDDEN_DIM,
                                                                          goal_pricerange_dim = GOAL_PRICERANGE_DIM)

model_LecTrackGoalPricerangeClassifier = model_LecTrackGoalPricerangeClassifier.to(DEVICE)

optimizer_LecTrackGoalPricerangeClassifier = optim.Adam(model_LecTrackGoalPricerangeClassifier.parameters(), lr = 1e-4)

In [ ]:
class LecTrackGoalNameClassifier(nn.Module):
    
    def __init__(self, hidden_dim, goal_name_dim):
        super(LecTrackGoalNameClassifier, self).__init__()
        self.goal_name_dim = goal_name_dim
        self.linear = nn.Linear(in_features = hidden_dim,
                                out_features = goal_name_dim)
        
    def forward(self, hidden):
        return F.log_softmax(self.linear(hidden).view(-1, self.goal_name_dim), dim = 1)
    
model_LecTrackGoalNameClassifier = LecTrackGoalNameClassifier(hidden_dim = HIDDEN_DIM,
                                                             goal_name_dim = GOAL_NAME_DIM)

model_LecTrackGoalNameClassifier = model_LecTrackGoalNameClassifier.to(DEVICE)

optimizer_LecTrackGoalNameClassifier = optim.Adam(model_LecTrackGoalNameClassifier.parameters(), lr = 1e-4)

In [ ]:
class LecTrackGoalAreaClassifier(nn.Module):
    
    def __init__(self, hidden_dim, goal_area_dim):
        super(LecTrackGoalAreaClassifier, self).__init__()
        self.goal_area_dim = goal_area_dim
        self.linear = nn.Linear(in_features = hidden_dim,
                                out_features = goal_area_dim)
        
    def forward(self, hidden):
        return F.log_softmax(self.linear(hidden).view(-1, self.goal_area_dim), dim = 1)

model_LecTrackGoalAreaClassifier = LecTrackGoalAreaClassifier(hidden_dim = HIDDEN_DIM,
                                                             goal_area_dim = GOAL_AREA_DIM)

model_LecTrackGoalAreaClassifier = model_LecTrackGoalAreaClassifier.to(DEVICE)

optimizer_LecTrackGoalAreaClassifier = optim.Adam(model_LecTrackGoalAreaClassifier.parameters(), lr = 1e-4)

In [ ]:
def get_index_and_score(turn, token_to_index, device):
    indices = []
    scores = []
    token_score_list = turn["system"] + turn["user"]
    for token, score in token_score_list:
        if token not in token_to_index:
            indices.append(token_to_index["<unk>"])
        else:
            indices.append(token_to_index[token])
        scores.append(score)
    assert len(indices) == len(scores)
    return torch.tensor(indices, dtype = torch.long, device = device), torch.tensor(scores, dtype = torch.float, device = device)

In [ ]:
def retrieve_gold_Method(raw_Y, ontology, device):
    ontology_methods = ontology["method"]
    raw_goal_method = raw_Y["method"]
    goal_method = ontology_methods.index(raw_goal_method)
    return torch.tensor([goal_method], dtype = torch.long, device = device)

def retrieve_gold_Requested(raw_Y, ontology, device):
    ontology_requestable = ontology["requestable"]
    raw_gold_requested = raw_Y["requested"]
    
    binary_representation = np.zeros(len(ontology_requestable), dtype = int)
    if len(raw_gold_requested) != 0:
        for requested in raw_gold_requested:
            binary_representation[ontology_requestable.index(requested)] = 1        
    
    goal_requested = binary_representation.dot(2**np.arange(binary_representation.size)[::-1])
    return torch.tensor([goal_requested], dtype = torch.long, device = device)

def retrieve_gold_GoalFood(raw_Y, ontology, device):
    ontology_informable_food = ontology["informable"]["food"]
    raw_goal_food = raw_Y["goal"]["food"]
    goal_food = 0
    if raw_goal_food != None:
        if raw_goal_food == "dontcare":
            goal_food = 1
        else:    
            goal_food = ontology_informable_food.index(raw_goal_food) + 2
    return torch.tensor([goal_food], dtype = torch.long, device = device)

def retrieve_gold_GoalPriceRange(raw_Y, ontology, device):
    ontology_informable_pricerange = ontology["informable"]["pricerange"]
    raw_goal_pricerange = raw_Y["goal"]["pricerange"]
    goal_pricerange = 0
    if raw_goal_pricerange != None:
        if raw_goal_pricerange == "dontcare":
            goal_pricerange = 1
        else:    
            goal_pricerange = ontology_informable_pricerange.index(raw_goal_pricerange) + 2
    return torch.tensor([goal_pricerange], dtype = torch.long, device = device)

def retrieve_gold_GoalName(raw_Y, ontology, device):
    ontology_informable_name = ontology["informable"]["name"]
    raw_goal_name = raw_Y["goal"]["name"]
    goal_name = 0
    if raw_goal_name != None:
        if raw_goal_name == "dontcare":
            goal_name = 1
        else:    
            goal_name = ontology_informable_name.index(raw_goal_name) + 2
    return torch.tensor([goal_name], dtype = torch.long, device = device)

def retrieve_gold_GoalArea(raw_Y, ontology, device):
    ontology_informable_area = ontology["informable"]["area"]
    raw_goal_area = raw_Y["goal"]["area"]
    goal_area = 0
    if raw_goal_area != None:
        if raw_goal_area == "dontcare":
            goal_area = 1
        else:    
            goal_area = ontology_informable_area.index(raw_goal_area) + 2
    return torch.tensor([goal_area], dtype = torch.long, device = device)

In [ ]:
train_epoch_losses = []
dev_epoch_losses = []

# define the loss function
loss_function = nn.CrossEntropyLoss()

for epoch in range(NUM_EPOCHS):
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    train_epoch_loss = 0
    
    model_LecTrackEncoder = model_LecTrackEncoder.train()
    model_LecTrackMethodClassifier = model_LecTrackMethodClassifier.train()
    model_LecTrackRequestedClassifier = model_LecTrackRequestedClassifier.train()
    model_LecTrackGoalFoodClassifier = model_LecTrackGoalFoodClassifier.train()
    model_LecTrackGoalPricerangeClassifier = model_LecTrackGoalPricerangeClassifier.train()
    model_LecTrackGoalNameClassifier = model_LecTrackGoalNameClassifier.train()
    model_LecTrackGoalAreaClassifier = model_LecTrackGoalAreaClassifier.train()
    
    # for each dialog
    for raw_X_train_dialog, raw_Y_train_dialog in tqdm(zip(raw_X_train, raw_Y_train), total = len(raw_X_train)):

        model_LecTrackEncoder.hidden = model_LecTrackEncoder.init_hidden()
        model_LecTrackEncoder.zero_grad()
        model_LecTrackMethodClassifier.zero_grad()
        model_LecTrackRequestedClassifier.zero_grad()
        model_LecTrackGoalFoodClassifier.zero_grad()
        model_LecTrackGoalPricerangeClassifier.zero_grad()
        model_LecTrackGoalNameClassifier.zero_grad()
        model_LecTrackGoalAreaClassifier.zero_grad()

        # for each turn in the dialog
        for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):

            indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, device = DEVICE)

            output_LecTrackEncoder = model_LecTrackEncoder(indices, scores)
            output_LecTrackEncoder = output_LecTrackEncoder[1]
            
            output_LecTrackMethodClassifier = model_LecTrackMethodClassifier(output_LecTrackEncoder)
            output_LecTrackRequestedClassifier = model_LecTrackRequestedClassifier(output_LecTrackEncoder)
            output_LecTrackGoalFoodClassifier = model_LecTrackGoalFoodClassifier(output_LecTrackEncoder)
            output_LecTrackGoalPricerangeClassifier = model_LecTrackGoalPricerangeClassifier(output_LecTrackEncoder)
            output_LecTrackGoalNameClassifier = model_LecTrackGoalNameClassifier(output_LecTrackEncoder)
            output_LecTrackGoalAreaClassifier = model_LecTrackGoalAreaClassifier(output_LecTrackEncoder)

            gold_LecTrackMethodClassifier = retrieve_gold_Method(raw_Y_train_turn, ontology, device = DEVICE)
            gold_LecTrackRequestedClassifier = retrieve_gold_Requested(raw_Y_train_turn, ontology, device = DEVICE)
            gold_LecTrackGoalFoodClassifier = retrieve_gold_GoalFood(raw_Y_train_turn, ontology, device = DEVICE)
            gold_LecTrackGoalPricerangeClassifier = retrieve_gold_GoalPriceRange(raw_Y_train_turn, ontology, device = DEVICE)
            gold_LecTrackGoalNameClassifier = retrieve_gold_GoalName(raw_Y_train_turn, ontology, device = DEVICE)
            gold_LecTrackGoalAreaClassifier = retrieve_gold_GoalArea(raw_Y_train_turn, ontology, device = DEVICE)

            loss_LecTrackMethodClassifier = loss_function(output_LecTrackMethodClassifier, gold_LecTrackMethodClassifier)
            loss_LecTrackRequestedClassifier = loss_function(output_LecTrackRequestedClassifier, gold_LecTrackRequestedClassifier)
            loss_LecTrackGoalFoodClassifier = loss_function(output_LecTrackGoalFoodClassifier, gold_LecTrackGoalFoodClassifier)
            loss_LecTrackGoalPricerangeClassifier = loss_function(output_LecTrackGoalPricerangeClassifier, gold_LecTrackGoalPricerangeClassifier)
            loss_LecTrackGoalNameClassifier = loss_function(output_LecTrackGoalNameClassifier, gold_LecTrackGoalNameClassifier)
            loss_LecTrackGoalAreaClassifier = loss_function(output_LecTrackGoalAreaClassifier, gold_LecTrackGoalAreaClassifier)

            loss = loss_LecTrackMethodClassifier + \
                    loss_LecTrackRequestedClassifier + \
                    loss_LecTrackGoalFoodClassifier + \
                    loss_LecTrackGoalPricerangeClassifier + \
                    loss_LecTrackGoalNameClassifier + \
                    loss_LecTrackGoalAreaClassifier
            loss.backward(retain_graph = True)
            
            train_epoch_loss += loss

            optimizer_LecTrackEncoder.step()
            optimizer_LecTrackMethodClassifier.step()
            optimizer_LecTrackRequestedClassifier.step()
            optimizer_LecTrackGoalFoodClassifier.step()
            optimizer_LecTrackGoalPricerangeClassifier.step()
            optimizer_LecTrackGoalNameClassifier.step()
            optimizer_LecTrackGoalAreaClassifier.step()
    
    train_epoch_losses.append(train_epoch_loss.item())
    
    model_LecTrackEncoder = model_LecTrackEncoder.eval()
    model_LecTrackMethodClassifier = model_LecTrackMethodClassifier.eval()
    model_LecTrackRequestedClassifier = model_LecTrackRequestedClassifier.eval()
    model_LecTrackGoalFoodClassifier = model_LecTrackGoalFoodClassifier.eval()
    model_LecTrackGoalPricerangeClassifier = model_LecTrackGoalPricerangeClassifier.eval()
    model_LecTrackGoalNameClassifier = model_LecTrackGoalNameClassifier.eval()
    model_LecTrackGoalAreaClassifier = model_LecTrackGoalAreaClassifier.eval()
    
    with torch.no_grad():
        
        dev_epoch_loss = 0
        
        for raw_X_dev_dialog, raw_Y_dev_dialog in tqdm(zip(raw_X_dev, raw_Y_dev), total = len(raw_X_dev)):
            
            for raw_X_dev_turn, raw_Y_dev_turn in zip(raw_X_dev_dialog["turns"], raw_Y_dev_dialog["turns"]):
                
                indices, scores = get_index_and_score(raw_X_dev_turn, token_to_index, device = DEVICE)
                
                output_LecTrackEncoder = model_LecTrackEncoder(indices, scores)
                output_LecTrackEncoder = output_LecTrackEncoder[1]
                
                output_LecTrackMethodClassifier = model_LecTrackMethodClassifier(output_LecTrackEncoder)
                output_LecTrackRequestedClassifier = model_LecTrackRequestedClassifier(output_LecTrackEncoder)
                output_LecTrackGoalFoodClassifier = model_LecTrackGoalFoodClassifier(output_LecTrackEncoder)
                output_LecTrackGoalPricerangeClassifier = model_LecTrackGoalPricerangeClassifier(output_LecTrackEncoder)
                output_LecTrackGoalNameClassifier = model_LecTrackGoalNameClassifier(output_LecTrackEncoder)
                output_LecTrackGoalAreaClassifier = model_LecTrackGoalAreaClassifier(output_LecTrackEncoder)
                
                gold_LecTrackMethodClassifier = retrieve_gold_Method(raw_Y_dev_turn, ontology, device = DEVICE)
                gold_LecTrackRequestedClassifier = retrieve_gold_Requested(raw_Y_dev_turn, ontology, device = DEVICE)
                gold_LecTrackGoalFoodClassifier = retrieve_gold_GoalFood(raw_Y_dev_turn, ontology, device = DEVICE)
                gold_LecTrackGoalPricerangeClassifier = retrieve_gold_GoalPriceRange(raw_Y_dev_turn, ontology, device = DEVICE)
                gold_LecTrackGoalNameClassifier = retrieve_gold_GoalName(raw_Y_dev_turn, ontology, device = DEVICE)
                gold_LecTrackGoalAreaClassifier = retrieve_gold_GoalArea(raw_Y_dev_turn, ontology, device = DEVICE)

                loss_LecTrackMethodClassifier = loss_function(output_LecTrackMethodClassifier, gold_LecTrackMethodClassifier)
                loss_LecTrackRequestedClassifier = loss_function(output_LecTrackRequestedClassifier, gold_LecTrackRequestedClassifier)
                loss_LecTrackGoalFoodClassifier = loss_function(output_LecTrackGoalFoodClassifier, gold_LecTrackGoalFoodClassifier)
                loss_LecTrackGoalPricerangeClassifier = loss_function(output_LecTrackGoalPricerangeClassifier, gold_LecTrackGoalPricerangeClassifier)
                loss_LecTrackGoalNameClassifier = loss_function(output_LecTrackGoalNameClassifier, gold_LecTrackGoalNameClassifier)
                loss_LecTrackGoalAreaClassifier = loss_function(output_LecTrackGoalAreaClassifier, gold_LecTrackGoalAreaClassifier)
                
                loss = loss_LecTrackMethodClassifier + \
                        loss_LecTrackRequestedClassifier + \
                        loss_LecTrackGoalFoodClassifier + \
                        loss_LecTrackGoalPricerangeClassifier + \
                        loss_LecTrackGoalNameClassifier + \
                        loss_LecTrackGoalAreaClassifier
                
                dev_epoch_loss += loss
                
        dev_epoch_losses.append(dev_epoch_loss.item())
            
    logging.info("Epoch train loss\t{} \tdev loss\t{}".format(train_epoch_loss, dev_epoch_loss))
    

In [ ]:
plotly.offline.iplot({
    "data": [Scatter(
                x = list(range(len(train_epoch_losses))),
                y = train_epoch_losses,
                mode = "lines+markers",
                name = "train loss",
                marker = dict(color = "#1abc9c")),
            Scatter(
                x = list(range(len(dev_epoch_losses))),
                y = dev_epoch_losses,
                mode = "lines+markers",
                name = "dev loss",
                marker = dict(color = "#3498db"))],
    "layout": Layout(title = "<b>Train-Dev Loss</b>",
                     xaxis = dict(title = "<b>Epoch</b>", dtick = 1, titlefont = dict(color = "#34495e")),
                     yaxis = dict(title = "<b>Loss</b>", titlefont = dict(color = "#34495e")),
                     margin = Margin(b = 150)
                    )
})

In [ ]:
get_bin = lambda x, n: format(x, "b").zfill(n)

def retrieve_output_Method(output_tensor):
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    method_dict = {}
    
    index = torch.argmax(output_tensor).item()
    method_dict[ontology["method"][index]] = output_tensor[index].item()
    
    #for index in range(len(output_tensor)):
    #    method_dict[ontology["method"][index]] = output_tensor[index].item()
    
    return method_dict

def retrieve_output_Requested(output_tensor):
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    requested_dict = {}
    binary_representation = get_bin(torch.argmax(output_tensor).item(), len(ontology["requestable"]))
    indices = []
    index = 0
    for value in binary_representation:
        if int(value) == 1:
            indices.append(index)
        index += 1
    for index in indices:
        requested_dict[ontology["requestable"][index]] = (1 / len(indices))
    return requested_dict

def retrieve_output_GoalFood(output_tensor):
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_food_dict = {}
    
    index = torch.argmax(output_tensor).item()
    if index != 0:
        if index == 1:
            goal_food_dict["dontcare"] = output_tensor[index].item() 
        else:
            goal_food_dict[ontology["informable"]["food"][index - 2]] = output_tensor[index].item()
        
    #if torch.argmax(output_tensor).item() != 0:
    #    goal_food_dict["dontcare"] = output_tensor[1].item()
    #    for index in range(2, len(output_tensor)):
    #        goal_food_dict[ontology["informable"]["food"][index - 2]] = output_tensor[index].item()
    
    return goal_food_dict

def retrieve_output_GoalPricerange(output_tensor):
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_pricerange_dict = {}
    
    index = torch.argmax(output_tensor).item()
    if index != 0:
        if index == 1:
            goal_pricerange_dict["dontcare"] = output_tensor[index].item()
        else:     
            goal_pricerange_dict[ontology["informable"]["pricerange"][index - 2]] = output_tensor[index].item()
        
    #if torch.argmax(output_tensor).item() != 0:
    #    goal_pricerange_dict["dontcare"] = output_tensor[1].item()
    #    for index in range(2, len(output_tensor)):
    #        goal_pricerange_dict[ontology["informable"]["pricerange"][index - 2]] = output_tensor[index].item()
    
    return goal_pricerange_dict

def retrieve_output_GoalName(output_tensor):
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_name_dict = {}
    
    index = torch.argmax(output_tensor).item()
    if index != 0:
        if index == 1:
            goal_name_dict["dontcare"] = output_tensor[index].item()
        else:    
            goal_name_dict[ontology["informable"]["name"][index - 2]] = output_tensor[index].item()
        
    #if torch.argmax(output_tensor).item() != 0:
    #    goal_name_dict["dontcare"] = output_tensor[1].item()
    #    for index in range(2, len(output_tensor)):
    #        goal_name_dict[ontology["informable"]["name"][index - 2]] = output_tensor[index].item()
    
    return goal_name_dict

def retrieve_output_GoalArea(output_tensor):
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_area_dict = {}
    
    index = torch.argmax(output_tensor).item()
    if index != 0:
        if index == 1:
            goal_area_dict["dontcare"] = output_tensor[index].item()
        else:
            goal_area_dict[ontology["informable"]["area"][index - 2]] = output_tensor[index].item()
        
    #if torch.argmax(output_tensor).item() != 0:
    #    goal_area_dict["dontcare"] = output_tensor[1].item()
    #    for index in range(2, len(output_tensor)):
    #        goal_area_dict[ontology["informable"]["area"][index - 2]] = output_tensor[index].item()
    
    return goal_area_dict

In [ ]:
def make_tracker(raw_X, raw_Y, dataset):
    with torch.no_grad():
        tracker_json = {}
        tracker_json["dataset"] = dataset
        tracker_json["sessions"] = []

        start_time = time.time()
        for raw_X_dialog, raw_Y_dialog in tqdm(zip(raw_X, raw_Y), total = len(raw_X)):

            session = {}
            session["session-id"] = raw_X_dialog["session-id"]
            session["turns"] = []

            for raw_X_turn, raw_Y_turn in zip(raw_X_dialog["turns"], raw_Y_dialog["turns"]):
                turn = {}
                turn["goal-labels"] = {}

                indices, scores = get_index_and_score(raw_X_turn, token_to_index, device = DEVICE)

                output_LecTrackEncoder = model_LecTrackEncoder(indices, scores)
                output_LecTrackEncoder = output_LecTrackEncoder[1]

                output_LecTrackMethodClassifier = model_LecTrackMethodClassifier(output_LecTrackEncoder)
                turn["method-label"] = retrieve_output_Method(output_LecTrackMethodClassifier)

                output_LecTrackRequestedClassifier = model_LecTrackRequestedClassifier(output_LecTrackEncoder)
                if torch.argmax(output_LecTrackRequestedClassifier).item() == 0:
                    turn["requested-slots"] = {}
                else:
                    turn["requested-slots"] = retrieve_output_Requested(output_LecTrackRequestedClassifier)

                output_LecTrackGoalFoodClassifier = model_LecTrackGoalFoodClassifier(output_LecTrackEncoder)
                if torch.argmax(output_LecTrackGoalFoodClassifier).item() != 0:
                    turn["goal-labels"]["food"] = retrieve_output_GoalFood(output_LecTrackGoalFoodClassifier)

                output_LecTrackGoalPricerangeClassifier = model_LecTrackGoalPricerangeClassifier(output_LecTrackEncoder)
                if torch.argmax(output_LecTrackGoalPricerangeClassifier).item() != 0:
                    turn["goal-labels"]["pricerange"] = retrieve_output_GoalPricerange(output_LecTrackGoalPricerangeClassifier)

                output_LecTrackGoalNameClassifier = model_LecTrackGoalNameClassifier(output_LecTrackEncoder)
                if torch.argmax(output_LecTrackGoalNameClassifier).item() != 0:
                    turn["goal-labels"]["name"] = retrieve_output_GoalName(output_LecTrackGoalNameClassifier)

                output_LecTrackGoalAreaClassifier = model_LecTrackGoalAreaClassifier(output_LecTrackEncoder)
                if torch.argmax(output_LecTrackGoalAreaClassifier).item() != 0:
                    turn["goal-labels"]["area"] = retrieve_output_GoalArea(output_LecTrackGoalAreaClassifier)

                session["turns"].append(turn)
            tracker_json["sessions"].append(session)
        end_time = time.time()
        tracker_json["wall-time"] = end_time - start_time
        return tracker_json

In [ ]:
dev_tracker = make_tracker(raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")

with open("dev_tracker.json", "w") as dev_tracker_file:
    json.dump(dev_tracker, dev_tracker_file)

!python2 dstc2/dstc2_scripts/score.py\
--dataset dstc2_dev\
--dataroot dstc2/dstc2_traindev/data\
--ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
--trackfile dev_tracker.json\
--scorefile dev_tracker.score.csv

!python2 dstc2/dstc2_scripts/report.py --scorefile dev_tracker.score.csv